
# Taller Práctico N.º 1  
## Unidad: Construcción de Interfaces Funcionales con Componentes Inteligentes  
**Entorno:** Python + Google Colab  
**Objetivo:** Construir una interfaz analítica interactiva (en Colab) con **componentes inteligentes** que consumen **datos reales** desde una **API pública**, generan **alertas automáticas**, muestran **gráficas dinámicas** y realizan una **predicción simple**.

---



## 0) Requisitos (Colab)
- Abrir este notebook en **Google Colab**.
- No requiere claves de API (usaremos **Open‑Meteo**, API pública y gratuita).
- Bibliotecas: `requests`, `pandas`, `matplotlib`, `scikit-learn`, `ipywidgets`.


In [ ]:

# Ejecuta esta celda para instalar/asegurar dependencias (en Colab suelen venir preinstaladas)
!pip -q install ipywidgets scikit-learn
from IPython.display import display
print("Entorno listo ✔")



## Actividad 1: Configurar el entorno y estructura base
**Meta:** Crear funciones y estructura de proyecto mínima para reutilización (componentes inteligentes).


In [ ]:

import requests, json, math, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['axes.grid'] = True

@dataclass
class City:
    name: str
    lat: float
    lon: float

# Algunas ciudades de ejemplo (puedes agregar más):
CITIES = [
    City("Quito", -0.1807, -78.4678),
    City("Guayaquil", -2.1700, -79.9224),
    City("Cuenca", -2.9006, -79.0045),
    City("Lima", -12.0464, -77.0428),
    City("Bogotá", 4.7110, -74.0721),
]
print("Estructura base lista ✔")



## Actividad 2: **Conexión a una API pública** (paso a paso)
Usaremos **Open‑Meteo** (no requiere clave). Flujo:
1. **Definir coordenadas** (latitud/longitud) de la ciudad seleccionada.  
2. **Construir la URL** con parámetros (temperatura, humedad, viento por hora).  
3. **Hacer la petición** con `requests.get(...)`.  
4. **Validar la respuesta** (`status_code == 200`).  
5. **Parsear JSON** a `dict`.  
6. **Cargar a DataFrame** y **limpiar** tipos de datos/fechas.  
7. **Explorar** con `head()` y dimensiones.


In [ ]:

from datetime import datetime

def build_open_meteo_url(lat, lon):
    base = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m",
        "timezone": "auto"
    }
    qs = "&".join([f"{k}={v}" for k,v in params.items()])
    return f"{base}?{qs}"

def fetch_weather_df(city):
    url = build_open_meteo_url(city.lat, city.lon)
    r = requests.get(url, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"Error API ({r.status_code}): {r.text[:200]}")
    data = r.json()
    hourly = data.get("hourly", {})
    df = pd.DataFrame(hourly)
    df["time"] = pd.to_datetime(df["time"])
    df = df.rename(columns={
        "temperature_2m": "temp_C",
        "relative_humidity_2m": "rh_pct",
        "wind_speed_10m": "wind_ms"
    })
    df["city"] = city.name
    return df

city = CITIES[0]  # Quito
df = fetch_weather_df(city)
df.head()



## Actividad 3: **Componentes inteligentes** (reglas y alertas)
Implementa lógica que **clasifique estados** y **genere recomendaciones** en función de los datos.


In [ ]:

def smart_indicators(row):
    alerts = []
    if row.temp_C >= 30:
        alerts.append("⚠️ Calor alto: sugerir hidratación y ambientes ventilados.")
    elif row.temp_C <= 15:
        alerts.append("🌡️ Temperatura baja: sugerir abrigo.")
    if row.rh_pct >= 80:
        alerts.append("💧 Humedad alta: riesgo de condensación, vigilar equipos.")
    elif row.rh_pct <= 30:
        alerts.append("💦 Aire seco: considerar humidificación.")
    if row.wind_ms >= 10:
        alerts.append("🌀 Viento fuerte: precaución en exteriores.")
    return alerts

df["alerts"] = df.apply(smart_indicators, axis=1)
df["n_alerts"] = df["alerts"].apply(len)
df[["time","temp_C","rh_pct","wind_ms","n_alerts"]].head()



## Actividad 4: **Visualización dinámica**
Grafica temperatura, humedad y viento en el tiempo. Identifica picos y patrones.


In [ ]:

ax = df.plot(x="time", y=["temp_C","rh_pct","wind_ms"], title=f"Serie horaria — {city.name}")
ax.set_xlabel("Tiempo"); ax.set_ylabel("Valor"); plt.show()

ax2 = df.plot(x="time", y="n_alerts", kind="bar", title=f"Número de alertas inteligentes por hora — {city.name}")
ax2.set_xlabel("Tiempo"); ax2.set_ylabel("Nº alertas"); plt.tight_layout(); plt.show()



## Actividad 5: **Interacción con el usuario** (ipywidgets)
Permite que el usuario cambie de ciudad y actualice la visualización y alertas.


In [ ]:

import ipywidgets as widgets
from IPython.display import clear_output, display

def run_dashboard(selected_city):
    clear_output(wait=True)
    print(f"Ciudad seleccionada: {selected_city.name}")
    dfl = fetch_weather_df(selected_city)
    dfl["alerts"] = dfl.apply(smart_indicators, axis=1)
    dfl["n_alerts"] = dfl["alerts"].apply(len)
    display(dfl.head())
    ax = dfl.plot(x="time", y=["temp_C","rh_pct","wind_ms"], title=f"Serie horaria — {selected_city.name}")
    ax.set_xlabel("Tiempo"); ax.set_ylabel("Valor"); plt.show()
    ax2 = dfl.plot(x="time", y="n_alerts", kind="bar", title=f"Nº de alertas inteligentes por hora — {selected_city.name}")
    ax2.set_xlabel("Tiempo"); ax2.set_ylabel("Nº alertas"); plt.tight_layout(); plt.show()

city_options = {c.name: c for c in CITIES}
dd = widgets.Dropdown(options=list(city_options.keys()), description="Ciudad:")
display(dd)

def on_change(change):
    if change['name'] == 'value':
        run_dashboard(city_options[change['new']])

dd.observe(on_change)
print("Interfaz interactiva lista ✔ Selecciona una ciudad en el menú.")



## Actividad 6: **API sencilla — Paso a paso detallado**
**Objetivo:** Consumir una API pública sin clave (**Open‑Meteo**) y dejar el flujo reproducible.

**Pasos:**  
1) **Identificar la API y el recurso**: endpoint `https://api.open-meteo.com/v1/forecast`  
2) **Definir parámetros**: `latitude`, `longitude`, `hourly`, `timezone`.  
3) **Construir la URL** con parámetros (querystring).  
4) **Enviar petición** `GET` con `requests.get(url)`.  
5) **Verificar respuesta**: código `200` y estructura JSON (`.json()`).  
6) **Normalizar datos** en `pandas.DataFrame`.  
7) **Convertir tipos** (fechas, numéricos).  
8) **Validar** con `df.info()`, `df.head()`.  
9) **Persistir** si es necesario (`to_csv`).  
10) **Reutilizar** en componentes inteligentes y gráficas.


In [ ]:

from datetime import datetime

# 1) y 2) Definir ciudad y parámetros
target_city = City("Quito", -0.1807, -78.4678)
params = {
    "latitude": target_city.lat,
    "longitude": target_city.lon,
    "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m",
    "timezone": "auto"
}

# 3) Construcción de URL
base = "https://api.open-meteo.com/v1/forecast"
qs = "&".join([f"{k}={v}" for k,v in params.items()])
url = f"{base}?{qs}"
print("URL construida:", url)

# 4) Enviar GET
resp = requests.get(url, timeout=30)
print("Status code:", resp.status_code)

# 5) Validar/parsear JSON
data = resp.json() if resp.status_code == 200 else {}
print("Claves principales:", list(data.keys()))

# 6) Normalizar a DataFrame
hourly = data.get("hourly", {})
df_api = pd.DataFrame(hourly)

# 7) Convertir tipos
df_api["time"] = pd.to_datetime(df_api["time"])
df_api = df_api.rename(columns={
    "temperature_2m": "temp_C",
    "relative_humidity_2m": "rh_pct",
    "wind_speed_10m": "wind_ms"
})

# 8) Validaciones rápidas
display(df_api.head())
display(df_api.info())

# 9) Persistencia (opcional)
# df_api.to_csv("weather_quito.csv", index=False)

print("API consumida y datos listos ✔")



## Actividad 7: **Predicción simple** (baseline)
Predicción de la temperatura futura (siguiente 6 horas) con **Regresión Lineal**.


In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

dff = df_api.copy().dropna().sort_values("time").reset_index(drop=True)
dff["t_idx"] = np.arange(len(dff))
X = dff[["t_idx"]].values
y = dff["temp_C"].values

if len(dff) > 10:
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    mae = mean_absolute_error(y, y_pred)
    print(f"MAE (baseline lineal): {mae:.3f} °C")

    future_idx = np.arange(len(dff), len(dff)+6).reshape(-1, 1)
    future_pred = model.predict(future_idx)
    for h, temp in enumerate(future_pred, 1):
        print(f"Predicción +{h}h: {temp:.2f} °C")

    plt.plot(dff["time"], y, label="Real")
    plt.plot(dff["time"], y_pred, label="Ajuste lineal")
    plt.legend(); plt.title("Ajuste lineal — temperatura")
    plt.xticks(rotation=45); plt.tight_layout(); plt.show()
else:
    print("Datos insuficientes para la predicción baseline.")



## Actividad 8: **Reporte breve**
- Describe tus **componentes inteligentes** y reglas de decisión.  
- Resume el **pipeline** de conexión a la API (1–10).  
- Incluye capturas de **gráficas** y **predicciones**.  
- Propón mejoras (XAI, modelos estacionales, ARIMA/LSTM, despliegue).
